In [2]:
#Test BLS.gov API
import bls
import requests
import json
#import prettytable
import pandas as pd

In [3]:
from pprint import pprint
from config import api_key
from census import Census

#c = Census(api_key, year=2013)
#help(c)

In [4]:
for yr in range(2012,2017,1):
    c = Census(api_key, year=yr)
#    print(yr)

    try:
        census_data = c.acs5.get(("NAME", "B19013_001E", 
                              "B01003_001E", 
                              "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                           "B25064_001E",
                            "B25088_002E",
                            "B23025_004E",
                            "B23025_005E"
                           ), {'for': 'zip code tabulation area:*'})

        # Convert to DataFrame
        census_pd = pd.DataFrame(census_data)

        census_pd = census_pd.rename(columns={"B01003_001E": "Total Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Median Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B25064_001E":"median_gross_rent",
                                      "B25088_002E": "median_monthly_owner_costs",
                                      "B23025_004E":"employment_employed",
                                      "B23025_005E":"employment_unemployed",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
        
        #print(census_pd.loc[census_pd["Zipcode"]==10055])
        #break
        census_pd.to_csv(f"Data/census_{yr}.csv")
    except:
        print("Some error")
        
#census_pd.head()

In [5]:
#import zip file
zip_data_df = pd.read_csv("Data/free-zipcode-database.csv")
#zip_df = zip_data_df [["Zipcode", "City", "State", "Lat", "Long", "LocationText"]]
#zip_df.head()

zip_df = zip_data_df.loc[zip_data_df["State"]!="PR", ["Zipcode", "City", "State", "Lat", "Long", "LocationText"]]
zip_df.head()

/Users/paulinapaul/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Zipcode,City,State,Lat,Long,LocationText
112,7093,MONITOR,NJ,40.78,-74.00,"Monitor, NJ"
113,7093,TAURUS,NJ,40.78,-74.00,"Taurus, NJ"
114,7675,WESTWOOD,NJ,40.98,-74.03,"Westwood, NJ"
115,7675,OLD TAPPAN,NJ,40.98,-74.03,"Old Tappan, NJ"
116,7675,RIVER VALE,NJ,40.98,-74.03,"River Vale, NJ"


In [6]:
#merge census yearly data into a single dataframe
l = ['2012', '2013', '2014','2015', '2016']

mylist=[]
myDict={}
for item in l:
    
    #dataframe_name = f"census_{item}"
    df = pd.read_csv(f"Data/census_{item}.csv")
    df['year']=item
    
    #clean census data
    df.dropna(how="any", inplace=True)
    
    #remove erroraneous values in the census data
    df = df[df["Median Household Income"]!=-666666666]
    
    #merge zip_df to get city names
    merged_df = pd.merge(df, zip_df, on="Zipcode", how="left")
    
    #group by city, state
    city_state_grp = merged_df.groupby(["City","State"])

    new_df = pd.DataFrame({#"City": city_state_grp["City"].unique,
                           "Median Age": city_state_grp["Median Age"].mean(),
                            "Total Population": city_state_grp["Total Population"].sum(),
                        "Median Income": city_state_grp["Median Household Income"].mean(),
                        "Employed": city_state_grp["employment_employed"].sum(),
                        "Unemployed": city_state_grp["employment_unemployed"].sum(),
                           "Monthly Rent": city_state_grp["median_gross_rent"].mean(),
                           "Owner Monthly Costs": city_state_grp["median_monthly_owner_costs"].mean()
        
        
        
    })
    
    #pprint(new_df.head(100))
    #break
    
    new_df.reset_index(inplace=True)
    
    mylist.append(new_df)

    
    
#mylist[1].loc[mylist[1]["City"]=="SAN DIEGO" ]

In [ ]:
print(len(mylist))    
#print(mylist[1].head())

mylist[4].loc[mylist[4]["City"]=="NEW YORK"]

In [7]:
#filyer records with population >= 200,000



5


,City,State,Median Age,Total Population,Median Income,Employed,Unemployed,Monthly Rent,Owner Monthly Costs
31853,NEW YORK,IA,43.000000,1303.0,54259.000000,576.0,36.0,591.0,1.019000e+03
31854,NEW YORK,NY,36.568889,1625513.0,95989.666667,888938.0,65280.0,1966.4,-1.481187e+07
31855,NEW YORK,TX,44.400000,3022.0,58824.000000,1317.0,88.0,-666666666.0,1.110000e+03


In [8]:
#calculate the rate
#listGrowth = []
#for i in range(0, len(mylist)-1):
    
#    P_gr=(mylist[i+1]['Total Population']-mylist[i]['Total Population'])/mylist[i]['Total Population']
#    print(i)
#   listGrowth.append(P_gr)
    
#listGrowth[0]

#pop_growth_df = pd.DataFrame({"Population Growth Rate (2012-2013)": listGrowth[0],
#              "Population Growth Rate (2013-2014)": listGrowth[1],
#              "Population Growth Rate (2014-2015)": listGrowth[2],
#              "Population Growth Rate (2015-2016)": listGrowth[3]
#             })
#op_growth_df.head(100)

In [19]:
#calculate the rate
Growth_df  = pd.DataFrame()

for i in range(0, len(mylist)-1):
    
    
    Growth_df["City"]= mylist[i]["City"]
    Growth_df["State"]= mylist[i]["State"]
    
    Growth_df["2012_pop"] = mylist[0]['Total Population']
    
    P_gr=(mylist[i+1]['Total Population']-mylist[i]['Total Population'])/mylist[i]['Total Population']
    #print(i)
    col_name = f"Pop Growth Rate (201{i+2}-201{i+3})"
    Growth_df[col_name] = P_gr
    
    I_gr=(mylist[i+1]['Median Income']-mylist[i]['Median Income'])/mylist[i]['Median Income']
    #print(i)
    inc_name = f"Median Income Growth Rate (201{i+2}-201{i+3})"
    Growth_df[inc_name] = I_gr
    
    R_gr=(mylist[i+1]['Monthly Rent']-mylist[i]['Monthly Rent'])/mylist[i]['Monthly Rent']
    #print(i)
    Rent_name = f"Monthly Rent Growth Rate (201{i+2}-201{i+3})"
    Growth_df[Rent_name] = R_gr
    
    E_gr=(mylist[i+1]['Employed']-mylist[i]['Employed'])/mylist[i]['Employed']
    #print(i)
    Employed_name = f"Employment Growth Rate (201{i+2}-201{i+3})"
    Growth_df[Employed_name] = E_gr
    
    UE_gr=(mylist[i+1]['Unemployed']-mylist[i]['Unemployed'])/mylist[i]['Unemployed']
    #print(i)
    Unemployed_name = f"Unemployment Growth Rate (201{i+2}-201{i+3})"
    Growth_df[Unemployed_name] = UE_gr

    C_gr=(mylist[i+1]['Owner Monthly Costs']-mylist[i]['Owner Monthly Costs'])/mylist[i]['Owner Monthly Costs']
    #print(i)
    Cost_name = f"Owner Monthly Costs Growth Rate (201{i+2}-201{i+3})"
    Growth_df[Cost_name] = C_gr
    
Growth_df.head(100)

,City,State,2012_pop,Pop Growth Rate (2012-2013),Median Income Growth Rate (2012-2013),Monthly Rent Growth Rate (2012-2013),Employment Growth Rate (2012-2013),Unemployment Growth Rate (2012-2013),Owner Monthly Costs Growth Rate (2012-2013),Pop Growth Rate (2013-2014),...,Monthly Rent Growth Rate (2014-2015),Employment Growth Rate (2014-2015),Unemployment Growth Rate (2014-2015),Owner Monthly Costs Growth Rate (2014-2015),Pop Growth Rate (2015-2016),Median Income Growth Rate (2015-2016),Monthly Rent Growth Rate (2015-2016),Employment Growth Rate (2015-2016),Unemployment Growth Rate (2015-2016),Owner Monthly Costs Growth Rate (2015-2016)
0,100 PALMS,CA,18405.0,0.020375,-0.092405,0.019763,0.039919,-0.032410,0.068905,0.008360,...,-0.061947,0.033597,-0.211449,-0.030172,-0.029722,0.036729,-4.986523e-02,-0.052655,-0.128188,-0.037333
1,1000 PALMS,CA,7541.0,-0.014852,-0.010395,0.063683,-0.094949,-0.026110,-0.050923,0.066227,...,-0.069992,0.063683,-0.361991,-0.115561,-0.046088,0.010429,-2.162630e-02,-0.055423,-0.095745,-0.093541
2,12 MILE,IN,1083.0,-0.049861,-0.037101,-0.038462,-0.184116,0.111111,-0.078544,-0.172012,...,-0.318602,0.168394,-0.058824,-0.027778,0.093607,-0.074043,-1.005531e+06,0.053215,0.541667,-0.082857
3,29 PALMS,CA,23642.0,0.002073,-0.052383,0.007709,0.025449,0.147436,-0.040404,0.017770,...,-0.006515,0.011268,-0.055892,-0.014731,0.157082,0.365851,2.366120e-01,0.040490,0.063481,-293169.648197
4,30TH STREET TRAIN STATION,PA,53002.0,-0.003811,-0.011765,0.018496,-0.050634,-0.047765,0.004068,0.010606,...,0.019722,0.024209,0.034689,0.068576,0.003143,0.062979,3.526735e-02,0.058633,-0.012139,-0.032494
5,7 CORNERS,VA,13343.0,-0.021659,0.006329,0.022853,-0.026704,-0.068548,-0.064082,0.055845,...,-0.003745,0.030036,0.057229,-0.014487,-0.063247,0.042035,4.436090e-02,-0.053715,-0.276353,0.047775
6,9 MILE POINT,LA,32218.0,-0.020765,-0.057436,0.035496,-0.029974,-0.046068,0.000000,-0.015373,...,0.011806,-0.034119,-0.172557,-0.015009,-0.990365,0.265003,-1.318553e-01,-0.991457,-0.986181,-0.309524
7,AARON,GA,2499.0,-0.096038,-0.089097,0.064156,-0.043519,-0.052083,0.171558,0.040283,...,0.091346,-0.003872,0.206522,-0.007775,12.526622,0.088024,-3.524229e-02,11.392614,11.702703,0.019589
8,AARON,KY,10354.0,-0.013425,0.016440,0.089212,-0.032051,0.078385,0.068056,0.003524,...,-0.053763,0.032485,-0.062663,0.081186,-0.780897,0.072506,6.174242e-01,-0.731009,-0.690808,0.179976
9,AARONSBURG,PA,1192.0,0.093121,0.063854,0.006270,0.083172,-0.064516,0.003636,-0.027629,...,-0.290608,-0.127698,-0.294118,-0.129918,8.592628,-0.432774,-1.682243e-01,6.397938,8.583333,-0.295222


In [21]:
Growth_df['5 Year Average Population Growth Rate'] = sum([Growth_df['Pop Growth Rate (2012-2013)'], 
                                                      Growth_df['Pop Growth Rate (2013-2014)'],
                                                      Growth_df['Pop Growth Rate (2014-2015)'],
                                                      Growth_df['Pop Growth Rate (2015-2016)']])/5

filtered_Growth_df = Growth_df.loc[Growth_df['2012_pop']>=200000]

filtered_Growth_df.sort_values(by="5 Year Average Population Growth Rate", ascending=False)

,City,State,2012_pop,Pop Growth Rate (2012-2013),Median Income Growth Rate (2012-2013),Monthly Rent Growth Rate (2012-2013),Employment Growth Rate (2012-2013),Unemployment Growth Rate (2012-2013),Owner Monthly Costs Growth Rate (2012-2013),Pop Growth Rate (2013-2014),...,Employment Growth Rate (2014-2015),Unemployment Growth Rate (2014-2015),Owner Monthly Costs Growth Rate (2014-2015),Pop Growth Rate (2015-2016),Median Income Growth Rate (2015-2016),Monthly Rent Growth Rate (2015-2016),Employment Growth Rate (2015-2016),Unemployment Growth Rate (2015-2016),Owner Monthly Costs Growth Rate (2015-2016),5 Year Average Population Growth Rate
32473,PRAIRIE CITY,IN,246947.0,-0.997473,0.376180,-0.243366,-0.997181,-0.998975,0.078093,464.703526,...,-0.943860,-0.923442,-0.499399,-0.906064,0.661937,3.504532e-01,-0.871861,-0.935262,0.639328,92.373310
18442,HONO,HI,266385.0,-0.993348,-0.550776,-0.615981,-0.994557,-0.995483,-0.505734,-0.522009,...,517.968571,533.875000,1.137012,-0.985304,0.043126,-4.091453e-01,-0.984645,-0.983057,-0.377532,83.635405
19284,IOWA CITY,IA,356356.0,-0.996363,-0.277626,-0.613194,-0.996561,-0.996694,-0.577105,-0.578704,...,238.188034,41.696429,0.654453,-0.995326,-0.095532,-5.098443e-01,-0.995926,-0.988289,-0.393526,35.960647
36755,SHINGLE HOLLOW,NC,804812.0,-0.999646,-0.258011,-0.348067,-0.999563,-0.999822,-0.471691,135.217544,...,-0.575176,-0.370836,-0.616428,-0.910592,0.452043,9.565217e-01,-0.919272,-0.946054,0.146465,26.561951
29477,ODESSA,NY,276620.0,-0.992929,0.790052,-0.056125,-0.992997,-0.997464,-0.350763,104.834867,...,-0.992692,-0.999309,-0.234208,22.708035,0.319494,1.239051e+00,19.510015,144.454545,0.697124,25.111209
1672,AVALON,CA,203248.0,-0.991601,0.214742,-0.203791,-0.990857,-0.994723,-0.245733,121.086116,...,-0.979498,-0.986833,0.810753,0.144649,-0.217045,-3.870493e-01,0.156960,-0.602740,-0.586995,23.851581
14603,GAIL,TX,270801.0,-0.996887,0.154775,-0.161474,-0.996758,-0.999036,-0.204453,-0.641756,...,-0.026667,1.000000,0.016373,117.200000,-0.425608,7.377778e-01,130.835616,833.000000,0.374226,23.140748
34758,ROZEL,KS,211618.0,-0.917086,0.337003,0.264260,-0.903230,-0.917137,0.589587,0.723299,...,-0.990796,-0.993428,-0.457216,108.788462,-0.039102,4.843049e-01,90.709459,125.666667,0.401176,21.520655
41424,VARNEY,KY,276506.0,-0.998137,-0.328799,-0.527144,-0.998571,-0.997785,-0.403730,31.697087,...,-0.983659,-1.000000,-0.341486,75.411504,1.513072,5.160643e-01,155.046296,inf,0.700000,21.027459
41068,UNIV CITY,MO,315003.0,-0.975194,0.955569,0.332107,-0.974824,-0.986062,-0.090237,-0.963655,...,122.595238,47.785714,1.161417,-0.858424,0.031227,-3.308108e-01,-0.874077,-0.923865,-0.176685,19.818714


In [ ]:
url = 'https://api.census.gov/data/2017/pep/population?get=POP,GEONAME&for=place:*&in=state:01&DATE=9&key='



In [ ]:
year_code = {

   '2017': 10,
   '2016': 9,
   '2015': 8,
   '2014': 7,
   '2013': 6,
   '2012': 5,
   '2011': 4,
   '2010': 3,
   '2009': 2,
   '2008': 1
}


state_code = {

   "Alabama": 1,
   "Alaska": 2,
   "Arizona": 4,
   "Arkansas": 5,
   "California": 6,
   "Colorado": 8,
   "Connecticut": 9,
   "Delaware": 10,
   "District of Columbia": 11,
   "Florida": 12,
   "Georgia": 13,
   "Hawaii": 15,
   "Idaho": 16,
   "Illinois": 17,
   "Indiana": 18,
   "Iowa": 19,
   "Kansas": 20,
   "Kentucky": 21,
   "Louisiana": 22,
   "Maine": 23,
   "Maryland": 24,
   "Massachusetts": 25,
   "Michigan": 26,
   "Minnesota": 27,
   "Mississippi": 28,
   "Missouri": 29,
   "Montana": 30,
   "Nebraska": 31,
   "Nevada": 32,
   "New Hampshire": 33,
   "New Jersey": 34,
   "New Mexico": 35,
   "New York": 36,
   "North Carolina": 37,
   "North Dakota": 38,
   "Ohio": 39,
   "Oklahoma": 40,
   "Oregon": 41,
   "Pennsylvania": 42,
   "Rhode Island": 44,
   "South Carolina": 45,
   "South Dakota": 46,
   "Tennessee": 47,
   "Texas": 48,
   "Utah": 49,
   "Vermont": 50,
   "Virginia": 51,
   "Washington": 53,
   "West Virginia": 54,
   "Wisconsin": 55,
   "Wyoming": 56
}